In [1]:
!pip install requests

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import requests
import datetime
from urllib.parse import urlencode

In [3]:
import base64

In [4]:
client_id = "c060a698be6449279c07f9f4a3c6e952"
client_secret = "92f9c0d577024d3e85ee90482b268e19"

In [19]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id =self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return{
            "Authorization":f"Basic {client_creds_b64}" 
        }
    
    def get_token_data(self):
        return {
        "grant_type":"client_credentials"
        }
    
    def perform_auth(self):
        # do a lookup for a token
        # this token is for future requests
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url,data=token_data,headers=token_headers)
        if r.status_code not in range(200,299):
            raise Exception("Could not authenticate client.")
            #return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token'] # state-like item(they expire with time)
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
    def get_resource(self,lookup_id,resource_type="albums",version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def search(self,query,search_type='artist'):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query,"type":search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self,_id):
        return self.get_resource(_id,resource_type='albums')
    
    def get_artist(self,_id):
        return self.get_resource(_id,resource_type='artists')
  

In [20]:
spotify = SpotifyAPI(client_id,client_secret)

In [21]:
spotify.perform_auth()

True

In [22]:
access_token = spotify.access_token
access_token

'BQDqbNzuyszA7CXkVF8HukbyDjoLOu4Esr1GJGcEhoHvmqP4ULY9wyPVzz0L1GrJJ5j4yx1cMfmNrzwhq18'

In [26]:
spotify.search("Hope", search_type="Track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Hope&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/15UsOTVnJzReFVN1VCnxy4'},
       'href': 'https://api.spotify.com/v1/artists/15UsOTVnJzReFVN1VCnxy4',
       'id': '15UsOTVnJzReFVN1VCnxy4',
       'name': 'XXXTENTACION',
       'type': 'artist',
       'uri': 'spotify:artist:15UsOTVnJzReFVN1VCnxy4'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS'

In [31]:
spotify.get_artist("137W8MRPWKqSmrBGDBFSop")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/137W8MRPWKqSmrBGDBFSop'},
 'followers': {'href': None, 'total': 9222983},
 'genres': ['hip hop',
  'pittsburgh rap',
  'pop rap',
  'rap',
  'southern hip hop',
  'trap'],
 'href': 'https://api.spotify.com/v1/artists/137W8MRPWKqSmrBGDBFSop',
 'id': '137W8MRPWKqSmrBGDBFSop',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/119b0e8f0ddd4cfc13945e69205e201750b6006c',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/52820589a9f9ba23ca8d8598109ba6e739d61f77',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/e3d91022edb1dfabecea149ba0d35a4ba5d3be50',
   'width': 160}],
 'name': 'Wiz Khalifa',
 'popularity': 87,
 'type': 'artist',
 'uri': 'spotify:artist:137W8MRPWKqSmrBGDBFSop'}

In [33]:
spotify.get_album("2Ti79nwTsont5ZHfdxIzAm")

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/15UsOTVnJzReFVN1VCnxy4'},
   'href': 'https://api.spotify.com/v1/artists/15UsOTVnJzReFVN1VCnxy4',
   'id': '15UsOTVnJzReFVN1VCnxy4',
   'name': 'XXXTENTACION',
   'type': 'artist',
   'uri': 'spotify:artist:15UsOTVnJzReFVN1VCnxy4'}],
 'available_markets': ['AD',
  'AE',
  'AL',
  'AR',
  'AT',
  'AU',
  'BA',
  'BE',
  'BG',
  'BH',
  'BO',
  'BR',
  'BY',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FR',
  'GB',
  'GR',
  'GT',
  'HK',
  'HN',
  'HR',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JO',
  'JP',
  'KW',
  'KZ',
  'LB',
  'LI',
  'LT',
  'LU',
  'LV',
  'MA',
  'MC',
  'MD',
  'ME',
  'MK',
  'MT',
  'MX',
  'MY',
  'NI',
  'NL',
  'NO',
  'NZ',
  'OM',
  'PA',
  'PE',
  'PH',
  'PL',
  'PS',
  'PT',
  'PY',
  'QA',
  'RO',
  'RS',
  'RU',
  'SA',
  'SE',
  'SG',
  'SI',
  'SK